TESSERACT-OCR

ORWELL

In [ ]:
# import module
from pdf2image import convert_from_path
 
 
# Store Pdf with convert_from_path function
images = convert_from_path('./book_pdfs/orwell_1984.pdf')

for i in range(len(images)):
   
      # Save pages as images in the pdf
    images[i].save('./book_images/orwell_1984' + '/page' + str(i) + '.jpg', 'JPEG')

import pytesseract
from PIL import Image
import pandas as pd

text_pages = []
for i in range(len(images)):
    image = Image.open('./book_images/orwell_1984' + '/page' + str(i) + '.jpg')
    text = pytesseract.image_to_string(image, lang='eng')
    text = text.replace("\n"," ")
    print(i)
    text_pages.append(text)

text_book_df = pd.DataFrame(text_pages,columns=['text'])

text_book_df.to_csv("./models/tesseract_84_text.csv",index=False)
text_book_df = pd.read_csv("./models/tesseract_84_text.csv")

DL

In [ ]:
# import module
from pdf2image import convert_from_path
 
 
# Store Pdf with convert_from_path function
images = convert_from_path('./book_pdfs/goodfellow_deep_learning.pdf')

for i in range(len(images)):
   
      # Save pages as images in the pdf
    images[i].save('./book_images/goodfellow_deep_learning' + '/page' + str(i) + '.jpg', 'JPEG')

import pytesseract
from PIL import Image
import pandas as pd

text_pages = []
for i in range(len(images)):
    image = Image.open('./book_images/goodfellow_deep_learning' + '/page' + str(i) + '.jpg')
    text = pytesseract.image_to_string(image, lang='eng')
    text = text.replace("\n"," ")
    print(i)
    text_pages.append(text)

after_intro = text_pages[15:]
before_reference = after_intro[:-65]

text_book_df = pd.DataFrame(before_reference,columns=['text'])

text_book_df.to_csv("./models/tesseract_dl_text.csv",index=False)
text_book_df = pd.read_csv("./models/tesseract_dl_text.csv")

EASY-OCR

ORWELL

In [ ]:
import easyocr

reader = easyocr.Reader(['en']) # this needs to run only once to load the model into memory

images_length = pd.read_csv("./models/tesseract_84_text.csv").shape[0]

text_pages = []
for i in range(images_length):
    text_list = reader.readtext('./book_images/orwell_1984' + '/page' + str(i) + '.jpg',detail=0)
    text = ''.join(text_list)
    text = text.replace("\n"," ")
    print(i)
    text_pages.append(text)

text_book_df = pd.DataFrame(text_pages,columns=['text'])

text_book_df.to_csv("./models/easy_84_text.csv",index=False)
text_book_df = pd.read_csv("./models/easy_84_text.csv")

CONVERT IMAGE PAGES TO TEXT PAGES AND SAVE TEXT BOOK

In [48]:
text_book_df = pd.read_csv("./models/tesseract_84_text.csv")

In [49]:
text_book_df

,text
0,ees ed = — =
1,Eric Arthur Blair (25 June 1903 - 21 January 1...
2,PART ONE 2 1984
3,"CHAPTER 1 I was a bright cold day in April, a..."
4,ures which had something to do with the produc...
...,...
389,of the Party as a DOUBLEPLUSGOOD DUCKSPEAKER i...
390,ical to Ingsoc could only be entertained in a ...
391,the passage of time the distinguishing charact...
392,"THAT TO SECURE THESE RIGHTS, GOVERNMENTS ARE I..."


In [6]:
from tqdm import tqdm
tqdm.pandas()
from gpt_api import get_embedding

In [51]:
text_book_df['embedding'] = text_book_df.text.progress_apply(lambda x: get_embedding(x))

100%|██████████| 394/394 [01:45<00:00,  3.75it/s]


In [52]:
text_book_df[['text','embedding']].to_json("./models/tesseract_84.json")

In [58]:
text_book_df = pd.read_json("./models/tesseract_84.json")

In [59]:
text_book_df

,text,embedding
0,ees ed = — =,"[0.0028095939000000003, 0.0055296663000000005,..."
1,Eric Arthur Blair (25 June 1903 - 21 January 1...,"[-0.0196088552, 0.0024448179, -0.0153701222000..."
2,PART ONE 2 1984,"[-0.0010082746, -0.0240596328, -0.0204334836, ..."
3,"CHAPTER 1 I was a bright cold day in April, a...","[-0.0174080674, -0.0059617143000000004, -0.015..."
4,ures which had something to do with the produc...,"[-0.0218209475, -0.014119437, 0.00236234300000..."
...,...,...
389,of the Party as a DOUBLEPLUSGOOD DUCKSPEAKER i...,"[-0.0030891225000000003, 0.0009054925, 0.02249..."
390,ical to Ingsoc could only be entertained in a ...,"[-0.0095675662, -0.004203015, 0.0114945071, -0..."
391,the passage of time the distinguishing charact...,"[-0.0008211457, -0.0047661318, 0.0059636184000..."
392,"THAT TO SECURE THESE RIGHTS, GOVERNMENTS ARE I...","[0.0036198599000000003, -0.0065559684, -0.0087..."


LOAD EMBEDDINGS DF

In [7]:
import numpy as np
import pandas as pd

In [8]:
def bring_k_most_similar(query,ocr,k):
    text_book_df = pd.read_json(f"./models/{ocr}_84.json")
    query_embedding = get_embedding(query)
    similarity = text_book_df['embedding'].apply(lambda x: np.dot(x,query_embedding))
    relevant_indexes = similarity.sort_values(ascending=False).iloc[0:k].index
    relevant_text = ' '.join(text_book_df.iloc[relevant_indexes]['text'].values)
    return relevant_text

In [9]:
from gpt_api import model_output

In [10]:
def book_whisperer(query,ocr,k=3):
    relevant_text = bring_k_most_similar(query,ocr,k)
    gpt_prompt = f"""
        Instructions
        - You are a text based search engine.
        - Summarize the following text to be shorter and more concise.
        - Keywords and summary should be relevant to answer the question.
        - Retain as much information as needed to answer the question at a later time.

        Question: {query} 
        Text: {relevant_text}
        Summary:
    """
    return model_output(gpt_prompt)

In [ ]:
book_whisperer("Which poster was suddenly appeared all over london?",ocr='tesseract').replace("\n","")

In [ ]:
book_whisperer("Which poster was suddenly appeared all over london?",ocr='easy').replace("\n","")

In [11]:
book_whisperer("Who is Winston Smith?",ocr='tesseract').replace("\n","")

"Winston Smith is a man in his mid-forties with a heavy black moustache and ruggedly handsome features. He is arrested for an indiscretion while producing a definitive edition of the poems of Kipling, which was allowing the word 'God' to remain at the end of a line."

In [12]:
book_whisperer("Who is Winston Smith?",ocr='easy').replace("\n","")

'Winston Smith is a man of about 45 years old with a black moustache and ruggedly handsome features. He is featured in a poster with the caption "Big Brother is Watching You". He is thirty-nine years old and has a varicose ulcer above his right ankle. He is in the Records Department during the Two Minutes Hate, where Emmanuel Goldstein, the Enemy of the People, is featured on the screen. He is sitting next to a small, sandy-haired woman and behind a girl with dark hair. He is shoeless and unshaven, and is speaking to Ampleforth, who is suspected of bearing a razor blade. Ampleforth has committed an indiscretion by allowing the word "God" to remain at the end of a line in a definitive edition of the poems of Kipling.'